In [105]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statistics import mode 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['Data_Dictionary.xlsx', 'new_merchant_transactions.csv', 'test.csv', 'merchants.csv', 'historical_transactions.csv', 'all.zip', 'train.csv', 'sample_submission.csv']


In [150]:
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")
# merchants_df = pd.read_csv("./input/merchants.csv")
# new_merchant_transactions_df = pd.read_csv("./input/new_merchant_transactions.csv", )
# historical_transactions_df = pd.read_csv("./input/historical_transactions.csv")

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
historical_transactions_df = reduce_mem_usage(historical_transactions_df)
new_merchant_transactions_df = reduce_mem_usage(new_merchant_transactions_df)
merchants_df = reduce_mem_usage(merchants_df)

Memory usage of dataframe is 3109.54 MB
Memory usage after optimization is: 1622.97 MB
Decreased by 47.8%
Memory usage of dataframe is 209.67 MB
Memory usage after optimization is: 169.08 MB
Decreased by 19.4%
Memory usage of dataframe is 56.18 MB
Memory usage after optimization is: 32.66 MB
Decreased by 41.9%


In [6]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [7]:
train_df.shape

(201917, 6)

In [8]:
test_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [107]:
test_df.shape

(123623, 5)

In [10]:
merchants_df.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057465,-0.057465,N,E,E,-0.400000,...,-2.250000,18.666667,6,-2.320000,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057465,-0.057465,N,E,E,-0.720000,...,-0.740000,1.291667,6,-0.570000,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057465,-0.057465,N,E,E,-82.129997,...,-82.129997,260.000000,2,-82.129997,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057465,-0.057465,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057465,-0.057465,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [11]:
merchants_df.shape

(334696, 22)

In [12]:
new_merchant_transactions_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,NaN,-1,29


In [13]:
historical_transactions_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [14]:
historical_transactions_df.isna().sum()

authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               178159
merchant_category_id          0
merchant_id              138481
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2652864
state_id                      0
subsector_id                  0
dtype: int64

In [15]:
historical_transactions_df['category_3'].value_counts()

A    15411747
B    11677522
C     1844933
Name: category_3, dtype: int64

In [16]:
new_merchant_transactions_df.isna().sum()

authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64

In [17]:
new_merchant_transactions_df['category_3'].value_counts()

A    922244
B    836178
C    148687
Name: category_3, dtype: int64

In [18]:
new_merchant_transactions_df.head(2)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19


In [19]:
#historical_transactions_df.isna().sum()

In [20]:
new_merchant_transactions_df['category_2'].fillna(1.0, inplace=True)
new_merchant_transactions_df['category_3'].fillna('A', inplace=True)

historical_transactions_df['category_2'].fillna(1.0, inplace=True)
historical_transactions_df['category_3'].fillna('A', inplace=True)

In [21]:
#del historical_transactions_df['category_2']
#del historical_transactions_df['category_3']

#del new_merchant_transactions_df['category_2']
#del new_merchant_transactions_df['category_3']

In [22]:
historical_transactions_df = historical_transactions_df.dropna()
new_merchant_transactions_df = new_merchant_transactions_df.dropna()

In [108]:
train_df.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913


In [110]:
test_df.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0


In [151]:
train_df['first_active_month'] = pd.to_datetime(train_df['first_active_month'])
test_df['first_active_month'] = pd.to_datetime(test_df['first_active_month'])

In [152]:
train_df.dtypes

first_active_month    datetime64[ns]
card_id                       object
feature_1                      int64
feature_2                      int64
feature_3                      int64
target                       float64
dtype: object

In [27]:
#hist_trans_df_group = historical_transactions_df.groupby('card_id')

In [28]:
historical_transactions_df.head(2)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16


In [29]:
new_merchant_transactions_df.head(2)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19


In [30]:
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions_df = binarize(historical_transactions_df)
new_merchant_transactions_df = binarize(new_merchant_transactions_df)

In [31]:
historical_transactions_df['category_3'] = historical_transactions_df['category_3'].astype('category').cat.codes
new_merchant_transactions_df['category_3'] = new_merchant_transactions_df['category_3'].astype('category').cat.codes

In [32]:
historical_transactions_df.head(2)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16


In [33]:
new_merchant_transactions_df.head(2)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_415bb3a509,107,0,1,1,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,1,C_ID_415bb3a509,140,0,1,1,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19


In [34]:
transactions = historical_transactions_df.append(new_merchant_transactions_df)

In [35]:
historical_transactions_df.shape

(28973880, 14)

In [36]:
new_merchant_transactions_df.shape

(1936815, 14)

In [37]:
transactions.shape

(30910695, 14)

In [38]:
del historical_transactions_df
del new_merchant_transactions_df

In [40]:
from scipy import stats

In [41]:
def aggregate_transactions(history):
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    agg_func = {
    
    'authorized_flag': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
    'category_1': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
    'category_2': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
    'category_3': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
    'merchant_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
    'merchant_category_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
    'state_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
    'city_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
    'subsector_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'month_lag': [ lambda x: tuple(stats.mode(x)[0])]
    }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

In [42]:
history = aggregate_transactions(transactions)

/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [43]:
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]

,card_id,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_city_id_nunique,...,hist_installments_std,hist_category_1_sum,hist_category_1_mean,hist_category_1_<lambda>,hist_subsector_id_nunique,hist_subsector_id_<lambda>,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_<lambda>,hist_month_lag_<lambda>
0,C_ID_00007093c1,151,31,"(M_ID_9400cf2342,)",-78.173653,-0.517706,1.507069,-0.728876,0.296638,5,...,0.760504,28,0.185430,"(0,)",13,"(19,)",116,0.768212,"(1,)","(-8,)"
1,C_ID_0001238066,148,90,"(M_ID_d17aabd756,)",-86.597160,-0.585116,0.768095,-0.740723,0.187366,19,...,1.601545,4,0.027027,"(0,)",19,"(19,)",145,0.979730,"(1,)","(-2,)"
2,C_ID_0001506ef0,67,29,"(M_ID_b1fc88154d,)",-35.333813,-0.527370,1.493545,-0.740491,0.469379,3,...,0.122169,0,0.000000,"(0,)",12,"(33,)",63,0.940299,"(1,)","(-2,)"
3,C_ID_0001793786,247,150,"(M_ID_923d57de8d,)",-37.015015,-0.149858,4.554145,-0.745405,0.850442,11,...,0.141116,2,0.008097,"(0,)",25,"(37,)",220,0.890688,"(1,)","(-1,)"
4,C_ID_000183fdda,155,84,"(M_ID_f9cfe0a43b,)",-75.428757,-0.486637,2.764788,-0.737892,0.508951,10,...,2.070500,4,0.025806,"(0,)",21,"(16,)",148,0.954839,"(1,)","(-1,)"


In [44]:
history.shape

(325540, 35)

In [115]:
history.head(2)

,card_id,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_city_id_nunique,...,hist_installments_std,hist_category_1_sum,hist_category_1_mean,hist_category_1_<lambda>,hist_subsector_id_nunique,hist_subsector_id_<lambda>,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_<lambda>,hist_month_lag_<lambda>
0,C_ID_00007093c1,151,31,"(M_ID_9400cf2342,)",-78.173653,-0.517706,1.507069,-0.728876,0.296638,5,...,0.760504,28,0.185430,"(0,)",13,"(19,)",116,0.768212,"(1,)","(-8,)"
1,C_ID_0001238066,148,90,"(M_ID_d17aabd756,)",-86.597160,-0.585116,0.768095,-0.740723,0.187366,19,...,1.601545,4,0.027027,"(0,)",19,"(19,)",145,0.979730,"(1,)","(-2,)"


In [170]:
train = pd.merge(train_df, history, on='card_id', how='left')
test = pd.merge(test_df, history, on='card_id', how='left')

In [171]:
train['hist_authorized_flag_<lambda>'] = train['hist_authorized_flag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_category_1_<lambda>'] = train['hist_category_1_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)

In [173]:
train['hist_category_2_<lambda>'] = train['hist_category_2_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train['hist_category_3_<lambda>'] = train['hist_category_3_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train['hist_merchant_id_<lambda>'] = train['hist_merchant_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train['hist_merchant_category_id_<lambda>'] = train['hist_merchant_category_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_state_id_<lambda>'] = train['hist_state_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_city_id_<lambda>'] = train['hist_city_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_subsector_id_<lambda>'] = train['hist_subsector_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_month_lag_<lambda>'] = train['hist_month_lag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)

In [174]:
test['hist_authorized_flag_<lambda>'] = test['hist_authorized_flag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_category_1_<lambda>'] = test['hist_category_1_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_category_2_<lambda>'] = test['hist_category_2_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test['hist_category_3_<lambda>'] = test['hist_category_3_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))

test['hist_merchant_id_<lambda>'] = test['hist_merchant_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test['hist_merchant_category_id_<lambda>'] = test['hist_merchant_category_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_state_id_<lambda>'] = test['hist_state_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_city_id_<lambda>'] = test['hist_city_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_subsector_id_<lambda>'] = test['hist_subsector_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_month_lag_<lambda>'] = test['hist_month_lag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)

In [175]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,...,hist_installments_std,hist_category_1_sum,hist_category_1_mean,hist_category_1_<lambda>,hist_subsector_id_nunique,hist_subsector_id_<lambda>,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_<lambda>,hist_month_lag_<lambda>
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,2.027576,23,0.323944,0,13,29,47,0.661972,1,-6
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,...,0.437179,4,0.045977,0,15,33,86,0.988506,1,-10
2,2017-08-01,C_ID_b709037bc5,5,1,1,15,11,'M_ID_c708cbb89a',4.708528,0.313902,...,3.885259,2,0.133333,0,7,20,11,0.733333,1,-3
3,2017-12-01,C_ID_d27d835a9f,2,1,0,36,33,'M_ID_44485d60c0',-19.435101,-0.539864,...,2.205872,1,0.027778,0,14,27,36,1.000000,1,-2
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,116,53,'M_ID_d895cb84ae',37.205791,0.320740,...,0.478237,0,0.000000,0,15,33,93,0.801724,1,0


In [161]:
test.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'hist_transactions_count', 'hist_merchant_id_nunique',
       'hist_merchant_id_<lambda>', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_city_id_nunique', 'hist_city_id_<lambda>',
       'hist_merchant_category_id_nunique',
       'hist_merchant_category_id_<lambda>', 'hist_category_2_sum',
       'hist_category_2_mean', 'hist_category_2_<lambda>',
       'hist_state_id_nunique', 'hist_state_id_<lambda>',
       'hist_category_3_sum', 'hist_category_3_mean',
       'hist_category_3_<lambda>', 'hist_installments_sum',
       'hist_installments_mean', 'hist_installments_max',
       'hist_installments_min', 'hist_installments_std', 'hist_category_1_sum',
       'hist_category_1_mean', 'hist_category_1_<lambda>',
       'hist_subsector_id_nunique', 'hist_subsector_id_<lambda>',
      

In [124]:
 test.columns = ['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'hist_transactions_count', 'hist_authorized_flag_sum',
       'hist_authorized_flag_mean', 'hist_authorized_flag_mode',
       'hist_category_1_sum', 'hist_category_1_mean',
       'hist_category_1_mode', 'hist_category_2_sum',
       'hist_category_2_mean', 'hist_category_2_mode',
       'hist_category_3_sum', 'hist_category_3_mean',
       'hist_category_3_mode', 'hist_merchant_id_nunique',
       'hist_merchant_id_mode', 'hist_merchant_category_id_nunique',
       'hist_merchant_category_id_mode', 'hist_state_id_nunique',
       'hist_state_id_mode', 'hist_city_id_nunique',
       'hist_city_id_mode', 'hist_subsector_id_nunique',
       'hist_subsector_id_mode', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_installments_sum', 'hist_installments_mean',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_std', 'hist_month_lag_mode']

In [165]:
train.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_transactions_count', 'hist_merchant_id_nunique',
       'hist_merchant_id_<lambda>', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_city_id_nunique', 'hist_city_id_<lambda>',
       'hist_merchant_category_id_nunique',
       'hist_merchant_category_id_<lambda>', 'hist_category_2_sum',
       'hist_category_2_mean', 'hist_category_2_<lambda>',
       'hist_state_id_nunique', 'hist_state_id_<lambda>',
       'hist_category_3_sum', 'hist_category_3_mean',
       'hist_category_3_<lambda>', 'hist_installments_sum',
       'hist_installments_mean', 'hist_installments_max',
       'hist_installments_min', 'hist_installments_std', 'hist_category_1_sum',
       'hist_category_1_mean', 'hist_category_1_<lambda>',
       'hist_subsector_id_nunique', 'hist_subsector_id_<lambda

In [126]:
# train.columns = ['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
#        'target', 'hist_transactions_count', 'hist_authorized_flag_sum',
#        'hist_authorized_flag_mean', 'hist_authorized_flag_mode',
#        'hist_category_1_sum', 'hist_category_1_mean',
#        'hist_category_1_mode', 'hist_category_2_sum',
#        'hist_category_2_mean', 'hist_category_2_mode',
#        'hist_category_3_sum', 'hist_category_3_mean',
#        'hist_category_3_mode', 'hist_merchant_id_nunique',
#        'hist_merchant_id_mode', 'hist_merchant_category_id_nunique',
#        'hist_merchant_category_id_<lambda>', 'hist_state_id_nunique',
#        'hist_state_id_mode', 'hist_city_id_nunique',
#        'hist_city_id_mode', 'hist_subsector_id_nunique',
#        'hist_subsector_id_mode', 'hist_purchase_amount_sum',
#        'hist_purchase_amount_mean', 'hist_purchase_amount_max',
#        'hist_purchase_amount_min', 'hist_purchase_amount_std',
#        'hist_installments_sum', 'hist_installments_mean',
#        'hist_installments_max', 'hist_installments_min',
#        'hist_installments_std', 'hist_month_lag_mode']

In [127]:
train.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_mode,...,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_month_lag_mode
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,277,117,'M_ID_1a81c358a3',-175.047455,...,0.119513,0,0.000000,0,21,34,265,0.956679,1,-2
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,356,148,'M_ID_940fb4498f',-214.361801,...,1.499641,31,0.087079,0,24,34,345,0.969101,1,0


In [128]:
test.head(2)

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_mode,hist_category_1_sum,...,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_month_lag_mode
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,2.027576,23,0.323944,0,13,29,47,0.661972,1,-6
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,...,0.437179,4,0.045977,0,15,33,86,0.988506,1,-10


In [130]:
# del transactions

In [131]:
merchants_df.isna().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [132]:
merchants_df['category_2'].value_counts()

1.0    160888
5.0     52923
3.0     51887
4.0     36450
2.0     20661
Name: category_2, dtype: int64

In [60]:
#merchants_df['avg_sales_lag6'].value_counts()

In [61]:
###### merchants_df['avg_sales_lag3'].fillna(1.0, inplace = True)
merchants_df['avg_sales_lag3'].fillna(1.0, inplace = True)
merchants_df['avg_sales_lag6'].fillna(1.0, inplace = True)
merchants_df['avg_sales_lag12'].fillna(1.0, inplace = True)

In [62]:
merchants_df['category_2'].fillna(1.0, inplace = True)

In [63]:
merchants_df.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057465,-0.057465,N,E,E,-0.400000,...,-2.250000,18.666667,6,-2.320000,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057465,-0.057465,N,E,E,-0.720000,...,-0.740000,1.291667,6,-0.570000,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057465,-0.057465,N,E,E,-82.129997,...,-82.129997,260.000000,2,-82.129997,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057465,-0.057465,Y,E,E,1.000000,...,1.000000,4.666667,6,1.000000,3.833333,12,Y,-1,-1,1.0
4,M_ID_64456c37ce,2228,222,21,-0.057465,-0.057465,Y,E,E,1.000000,...,1.000000,0.361111,6,1.000000,0.347222,12,Y,-1,-1,1.0


In [64]:
merchants_df['category_4'].value_counts()

N    238596
Y     96100
Name: category_4, dtype: int64

In [65]:
merchants_df['category_1'] = merchants_df['category_1'].astype('category').cat.codes
merchants_df['most_recent_sales_range'] = merchants_df['most_recent_sales_range'].astype('category').cat.codes

merchants_df['most_recent_purchases_range'] = merchants_df['most_recent_purchases_range'].astype('category').cat.codes
merchants_df['category_4'] = merchants_df['category_4'].astype('category').cat.codes

In [66]:
merchants_df.apply(lambda x:len(x.unique()))

merchant_id                    334633
merchant_group_id              109391
merchant_category_id              324
subsector_id                       41
numerical_1                       950
numerical_2                       944
category_1                          2
most_recent_sales_range             5
most_recent_purchases_range         5
avg_sales_lag3                   3372
avg_purchases_lag3             100003
active_months_lag3                  3
avg_sales_lag6                   4507
avg_purchases_lag6             135202
active_months_lag6                  6
avg_sales_lag12                  5009
avg_purchases_lag12            172917
active_months_lag12                12
category_4                          2
city_id                           271
state_id                           25
category_2                          5
dtype: int64

In [67]:
merchants_df.columns

Index(['merchant_id', 'merchant_group_id', 'merchant_category_id',
       'subsector_id', 'numerical_1', 'numerical_2', 'category_1',
       'most_recent_sales_range', 'most_recent_purchases_range',
       'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3',
       'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6',
       'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12',
       'category_4', 'city_id', 'state_id', 'category_2'],
      dtype='object')

In [68]:
merchants_df.apply(lambda x:len(x.unique()))

merchant_id                    334633
merchant_group_id              109391
merchant_category_id              324
subsector_id                       41
numerical_1                       950
numerical_2                       944
category_1                          2
most_recent_sales_range             5
most_recent_purchases_range         5
avg_sales_lag3                   3372
avg_purchases_lag3             100003
active_months_lag3                  3
avg_sales_lag6                   4507
avg_purchases_lag6             135202
active_months_lag6                  6
avg_sales_lag12                  5009
avg_purchases_lag12            172917
active_months_lag12                12
category_4                          2
city_id                           271
state_id                           25
category_2                          5
dtype: int64

In [69]:
def aggregate_merchants(history):
    
    agg_func = {
    
        'merchant_group_id': [lambda x: tuple(stats.mode(x)[0])],
        'merchant_category_id': [lambda x: tuple(stats.mode(x)[0])],
        'subsector_id': [lambda x: tuple(stats.mode(x)[0])],
        'numerical_1': ['sum', 'mean'],
        'numerical_2': ['sum', 'mean'],
        'category_1': [lambda x: tuple(stats.mode(x)[0])],
        'most_recent_sales_range': [lambda x: tuple(stats.mode(x)[0])],
        'most_recent_purchases_range': [lambda x: tuple(stats.mode(x)[0])],
        'avg_sales_lag3': ['sum', 'mean'],
        'avg_purchases_lag3': ['sum', 'mean'],
        'active_months_lag3': [lambda x: tuple(stats.mode(x)[0])],
        'avg_sales_lag6': ['mean',lambda x: tuple(stats.mode(x)[0])],
        'avg_purchases_lag6': ['sum', 'mean'],
        'active_months_lag6': [lambda x: tuple(stats.mode(x)[0])],
        'avg_sales_lag12': ['sum', 'mean'],
        'avg_purchases_lag12': ['sum', 'mean'],
        'active_months_lag12': [lambda x: tuple(stats.mode(x)[0])],
        'category_4': [lambda x: tuple(stats.mode(x)[0])],
        'city_id': [lambda x: tuple(stats.mode(x)[0])],
        'state_id': [lambda x: tuple(stats.mode(x)[0])],
        'category_2': [lambda x: tuple(stats.mode(x)[0])]
        
    }
    
    agg_history = history.groupby(['merchant_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('merchant_id')
          .size()
          .reset_index(name='merchant_count'))
    
    agg_history = pd.merge(df, agg_history, on='merchant_id', how='left')
    
    return agg_history

In [70]:
merchants_data = aggregate_merchants(merchants_df)

In [71]:
merchants_data.columns = ['merch_' + c if c != 'merchant_id' else c for c in merchants_data.columns]
merchants_data[:5]

,merchant_id,merch_merchant_count,merch_merchant_group_id_<lambda>,merch_state_id_<lambda>,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_category_4_<lambda>,merch_numerical_2_sum,merch_numerical_2_mean,merch_active_months_lag6_<lambda>,...,merch_city_id_<lambda>,merch_category_1_<lambda>,merch_subsector_id_<lambda>,merch_merchant_category_id_<lambda>,merch_numerical_1_sum,merch_numerical_1_mean,merch_active_months_lag3_<lambda>,merch_avg_purchases_lag3_sum,merch_avg_purchases_lag3_mean,merch_most_recent_purchases_range_<lambda>
0,M_ID_000025127f,1,"(14602,)","(9,)",1.08,1.08,"(1,)",-0.057465,-0.057465,"(6,)",...,"(69,)","(0,)","(37,)","(80,)",-0.057465,-0.057465,"(3,)",1.880342,1.880342,"(4,)"
1,M_ID_0000699140,1,"(19420,)","(9,)",1.24,1.24,"(0,)",-0.057465,-0.057465,"(6,)",...,"(48,)","(0,)","(27,)","(87,)",-0.057465,-0.057465,"(3,)",0.997832,0.997832,"(3,)"
2,M_ID_00006a5552,1,"(52848,)","(15,)",1.22,1.22,"(0,)",-0.057465,-0.057465,"(6,)",...,"(64,)","(0,)","(29,)","(178,)",-0.057465,-0.057465,"(3,)",1.000000,1.000000,"(4,)"
3,M_ID_000087311e,1,"(44655,)","(-1,)",0.78,0.78,"(1,)",-0.057465,-0.057465,"(6,)",...,"(-1,)","(1,)","(1,)","(206,)",-0.057465,-0.057465,"(3,)",1.314815,1.314815,"(4,)"
4,M_ID_0000ab0b2d,1,"(35761,)","(9,)",1.06,1.06,"(0,)",-0.057465,-0.057465,"(6,)",...,"(-1,)","(0,)","(16,)","(367,)",-0.057465,-0.057465,"(3,)",1.089897,1.089897,"(3,)"


In [72]:
merchants_data.shape

(334633, 31)

In [133]:
merchants_data.columns

Index(['merchant_id', 'merch_merchant_count', 'merch_merchant_group_id_mode',
       'merch_merchant_category_id_mode', 'merch_subsector_id_mode',
       'merch_numerical_1_sum', 'merch_numerical_1_mean',
       'merch_numerical_2_sum', 'merch_numerical_2_mean',
       'merch_category_1_mode', 'merch_most_recent_sales_range_mode',
       'merch_most_recent_purchases_range_mode', 'merch_avg_sales_lag3_sum',
       'merch_avg_sales_lag3_mean', 'merch_avg_purchases_lag3_sum',
       'merch_avg_purchases_lag3_mean', 'merch_active_months_lag3_mode',
       'merch_avg_sales_lag6_mean', 'merch_avg_sales_lag6_mode',
       'merch_avg_purchases_lag6_sum', 'merch_avg_purchases_lag6_mean',
       'merch_active_months_lag6_mode', 'merch_avg_sales_lag12_sum',
       'merch_avg_sales_lag12_mean', 'merch_avg_purchases_lag12_sum',
       'merch_avg_purchases_lag12_mean', 'merch_active_months_lag12_mode',
       'merch_category_4_mode', 'merch_city_id_mode', 'merch_state_id_mode',
       'merch_categor

In [134]:
merchants_data.columns = ['merchant_id', 'merch_merchant_count',
       'merch_merchant_group_id_mode',
       'merch_merchant_category_id_mode', 'merch_subsector_id_mode',
       'merch_numerical_1_sum', 'merch_numerical_1_mean',
       'merch_numerical_2_sum', 'merch_numerical_2_mean',
       'merch_category_1_mode', 'merch_most_recent_sales_range_mode',
       'merch_most_recent_purchases_range_mode',
       'merch_avg_sales_lag3_sum', 'merch_avg_sales_lag3_mean',
       'merch_avg_purchases_lag3_sum', 'merch_avg_purchases_lag3_mean',
       'merch_active_months_lag3_mode', 'merch_avg_sales_lag6_mean',
       'merch_avg_sales_lag6_mode', 'merch_avg_purchases_lag6_sum',
       'merch_avg_purchases_lag6_mean', 'merch_active_months_lag6_mode',
       'merch_avg_sales_lag12_sum', 'merch_avg_sales_lag12_mean',
       'merch_avg_purchases_lag12_sum', 'merch_avg_purchases_lag12_mean',
       'merch_active_months_lag12_mode', 'merch_category_4_mode',
       'merch_city_id_mode', 'merch_state_id_mode',
       'merch_category_2_mode']

In [135]:
merchants_data.head()

,merchant_id,merch_merchant_count,merch_merchant_group_id_mode,merch_merchant_category_id_mode,merch_subsector_id_mode,merch_numerical_1_sum,merch_numerical_1_mean,merch_numerical_2_sum,merch_numerical_2_mean,merch_category_1_mode,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,M_ID_000025127f,1,"(14602,)","(9,)",1.08,1.08,"(1,)",-0.057465,-0.057465,"(6,)",...,"(69,)","(0,)","(37,)","(80,)",-0.057465,-0.057465,"(3,)",1.880342,1.880342,"(4,)"
1,M_ID_0000699140,1,"(19420,)","(9,)",1.24,1.24,"(0,)",-0.057465,-0.057465,"(6,)",...,"(48,)","(0,)","(27,)","(87,)",-0.057465,-0.057465,"(3,)",0.997832,0.997832,"(3,)"
2,M_ID_00006a5552,1,"(52848,)","(15,)",1.22,1.22,"(0,)",-0.057465,-0.057465,"(6,)",...,"(64,)","(0,)","(29,)","(178,)",-0.057465,-0.057465,"(3,)",1.000000,1.000000,"(4,)"
3,M_ID_000087311e,1,"(44655,)","(-1,)",0.78,0.78,"(1,)",-0.057465,-0.057465,"(6,)",...,"(-1,)","(1,)","(1,)","(206,)",-0.057465,-0.057465,"(3,)",1.314815,1.314815,"(4,)"
4,M_ID_0000ab0b2d,1,"(35761,)","(9,)",1.06,1.06,"(0,)",-0.057465,-0.057465,"(6,)",...,"(-1,)","(0,)","(16,)","(367,)",-0.057465,-0.057465,"(3,)",1.089897,1.089897,"(3,)"


In [136]:
del merchants_df

In [137]:
merchants_data.isna().sum()

merchant_id                               0
merch_merchant_count                      0
merch_merchant_group_id_mode              0
merch_merchant_category_id_mode           0
merch_subsector_id_mode                   0
merch_numerical_1_sum                     0
merch_numerical_1_mean                    0
merch_numerical_2_sum                     0
merch_numerical_2_mean                    0
merch_category_1_mode                     0
merch_most_recent_sales_range_mode        0
merch_most_recent_purchases_range_mode    0
merch_avg_sales_lag3_sum                  0
merch_avg_sales_lag3_mean                 0
merch_avg_purchases_lag3_sum              0
merch_avg_purchases_lag3_mean             0
merch_active_months_lag3_mode             0
merch_avg_sales_lag6_mean                 0
merch_avg_sales_lag6_mode                 0
merch_avg_purchases_lag6_sum              0
merch_avg_purchases_lag6_mean             0
merch_active_months_lag6_mode             0
merch_avg_sales_lag12_sum       

In [177]:
train['hist_merchant_id_<lambda>']

0         'M_ID_1a81c358a3'
1         'M_ID_940fb4498f'
2         'M_ID_5634fd83e0'
3         'M_ID_00a6ca8a8a'
4         'M_ID_cecefd9589'
5         'M_ID_12637a44e9'
6         'M_ID_548f18cdd2'
7         'M_ID_00a6ca8a8a'
8         'M_ID_124f356ec7'
9         'M_ID_6cb0602e94'
10        'M_ID_a30f2f4685'
11        'M_ID_2c049b075c'
12        'M_ID_1d56499c64'
13        'M_ID_0c913b8352'
14        'M_ID_50f575c681'
15        'M_ID_ae16956a28'
16        'M_ID_57df19bf28'
17        'M_ID_6c86c6f9b9'
18        'M_ID_25ae9d5e52'
19        'M_ID_e5374dabc0'
20        'M_ID_1d2ad7faef'
21        'M_ID_5a0a412718'
22        'M_ID_5c59739332'
23        'M_ID_00a6ca8a8a'
24        'M_ID_9e84cda3b1'
25        'M_ID_1d8085cf5d'
26        'M_ID_f7e47257c9'
27        'M_ID_b680418f90'
28        'M_ID_cfd42d87ed'
29        'M_ID_56e0c47cb1'
                ...        
201887    'M_ID_7bedae37ad'
201888    'M_ID_87eb01dcdc'
201889    'M_ID_5ba019a379'
201890    'M_ID_0013d298b2'
201891    'M_ID_8c6c

In [180]:
train['merchant_id'] = train['hist_merchant_id_<lambda>'].str.replace('\W', '')
test['merchant_id'] = train['hist_merchant_id_<lambda>'].str.replace('\W', '')

In [80]:
# del train['hist_merchant_id_mode']
# del test['hist_merchant_id_mode']

In [205]:
test['old_index'] = range(0, len(test))

In [206]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,...,hist_category_1_sum,hist_category_1_mean,hist_category_1_<lambda>,hist_subsector_id_nunique,hist_subsector_id_<lambda>,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_<lambda>,hist_month_lag_<lambda>,merchant_id
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,277,117,'M_ID_1a81c358a3',-175.047455,...,0,0.000000,0,21,34,265,0.956679,1,-2,M_ID_1a81c358a3
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,356,148,'M_ID_940fb4498f',-214.361801,...,31,0.087079,0,24,34,345,0.969101,1,0,M_ID_940fb4498f
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,44,14,'M_ID_5634fd83e0',-29.867586,...,0,0.000000,0,8,33,42,0.954545,1,-10,M_ID_5634fd83e0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,84,57,'M_ID_00a6ca8a8a',-54.146149,...,13,0.154762,0,15,37,84,1.000000,1,-4,M_ID_00a6ca8a8a
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,168,102,'M_ID_cecefd9589',-67.878319,...,17,0.101190,0,18,37,163,0.970238,1,-1,M_ID_cecefd9589


In [207]:
merchants_data.merchant_id.dtype

CategoricalDtype(categories=['M_ID_000025127f', 'M_ID_0000699140', 'M_ID_00006a5552',
                  'M_ID_000087311e', 'M_ID_0000ab0b2d', 'M_ID_0000edb21f',
                  'M_ID_0000f3475c', 'M_ID_0000fd7caf', 'M_ID_0001c38687',
                  'M_ID_0002237144',
                  ...
                  'M_ID_fffd4df9cd', 'M_ID_fffd7c5ee2', 'M_ID_fffddcdb37',
                  'M_ID_fffe6eb555', 'M_ID_fffee4321d', 'M_ID_fffeeb852d',
                  'M_ID_fffef87522', 'M_ID_ffff0af8e7', 'M_ID_ffff655e2c',
                  'M_ID_ffffc28eaa'],
                 ordered=False)

In [208]:
train_final = pd.merge(train, merchants_data,  on=['merchant_id'] )
test_final = pd.merge(test, merchants_data,  on=['merchant_id'] )

In [209]:
train_final.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,277,117,'M_ID_1a81c358a3',-175.047455,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
1,2016-12-01,C_ID_59908e6dfe,3,1,1,-1.218335,235,88,'M_ID_1a81c358a3',-150.645996,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
2,2017-08-01,C_ID_c685bb7b50,3,2,1,-0.938059,45,4,'M_ID_1a81c358a3',-32.290592,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
3,2013-08-01,C_ID_69b673b48b,3,1,1,-1.514573,10,5,'M_ID_1a81c358a3',-7.382210,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
4,2017-12-01,C_ID_3c883b3c2d,5,1,1,-1.195605,46,43,'M_ID_1a81c358a3',-15.803577,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"


In [210]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,...,hist_category_1_sum,hist_category_1_mean,hist_category_1_<lambda>,hist_subsector_id_nunique,hist_subsector_id_<lambda>,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_<lambda>,hist_month_lag_<lambda>,merchant_id
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,277,117,'M_ID_1a81c358a3',-175.047455,...,0,0.000000,0,21,34,265,0.956679,1,-2,M_ID_1a81c358a3
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,356,148,'M_ID_940fb4498f',-214.361801,...,31,0.087079,0,24,34,345,0.969101,1,0,M_ID_940fb4498f
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,44,14,'M_ID_5634fd83e0',-29.867586,...,0,0.000000,0,8,33,42,0.954545,1,-10,M_ID_5634fd83e0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,84,57,'M_ID_00a6ca8a8a',-54.146149,...,13,0.154762,0,15,37,84,1.000000,1,-4,M_ID_00a6ca8a8a
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,168,102,'M_ID_cecefd9589',-67.878319,...,17,0.101190,0,18,37,163,0.970238,1,-1,M_ID_cecefd9589


In [211]:
train_final.shape

(201917, 71)

In [212]:
train.shape

(201917, 41)

In [213]:
test.shape

(123623, 41)

In [214]:
test_final.shape

(123623, 71)

In [215]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,...,hist_category_1_mean,hist_category_1_<lambda>,hist_subsector_id_nunique,hist_subsector_id_<lambda>,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_<lambda>,hist_month_lag_<lambda>,merchant_id,old_index
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,0.323944,0,13,29,47,0.661972,1,-6,M_ID_1a81c358a3,0
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,...,0.045977,0,15,33,86,0.988506,1,-10,M_ID_940fb4498f,1
2,2017-08-01,C_ID_b709037bc5,5,1,1,15,11,'M_ID_c708cbb89a',4.708528,0.313902,...,0.133333,0,7,20,11,0.733333,1,-3,M_ID_5634fd83e0,2
3,2017-12-01,C_ID_d27d835a9f,2,1,0,36,33,'M_ID_44485d60c0',-19.435101,-0.539864,...,0.027778,0,14,27,36,1.000000,1,-2,M_ID_00a6ca8a8a,3
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,116,53,'M_ID_d895cb84ae',37.205791,0.320740,...,0.000000,0,15,33,93,0.801724,1,0,M_ID_cecefd9589,4


In [216]:
test_final.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
1,2017-01-01,C_ID_1c1808be6e,4,3,0,51,9,'M_ID_9139332ccc',-30.865807,-0.605212,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
2,2016-09-01,C_ID_75ba50b276,5,2,1,488,104,'M_ID_2e4ebcd6ff',-316.905365,-0.649396,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
3,2017-10-01,C_ID_e6ef39161c,4,2,0,49,36,'M_ID_50d57cdf98',-19.691996,-0.401877,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
4,2016-12-01,C_ID_9f24441e2a,2,1,0,66,42,'M_ID_00a6ca8a8a',-43.088379,-0.652854,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"


In [217]:
test_final = test_final.sort_values(by=['old_index'])

In [218]:
test_final.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
5,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,...,"(69,)","(0,)","(34,)","(560,)",40.468750,40.468750,"(3,)",1.065682,1.065682,"(0,)"
186,2017-08-01,C_ID_b709037bc5,5,1,1,15,11,'M_ID_c708cbb89a',4.708528,0.313902,...,"(143,)","(0,)","(33,)","(705,)",0.150757,0.150757,"(3,)",1.007511,1.007511,"(0,)"
322,2017-12-01,C_ID_d27d835a9f,2,1,0,36,33,'M_ID_44485d60c0',-19.435101,-0.539864,...,"(-1,)","(1,)","(7,)","(511,)",-0.104980,-0.052490,"(3,)",13.654417,6.827208,"(0,)"
10333,2015-12-01,C_ID_2b5e3df5c2,5,1,1,116,53,'M_ID_d895cb84ae',37.205791,0.320740,...,"(17,)","(0,)","(37,)","(278,)",-0.047546,-0.047546,"(3,)",1.103070,1.103070,"(3,)"


In [219]:
test_final.index = test_final['old_index'].values

In [220]:
test_final.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,"(69,)","(0,)","(34,)","(560,)",-0.017807,-0.017807,"(3,)",0.979454,0.979454,"(1,)"
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,...,"(69,)","(0,)","(34,)","(560,)",40.468750,40.468750,"(3,)",1.065682,1.065682,"(0,)"
2,2017-08-01,C_ID_b709037bc5,5,1,1,15,11,'M_ID_c708cbb89a',4.708528,0.313902,...,"(143,)","(0,)","(33,)","(705,)",0.150757,0.150757,"(3,)",1.007511,1.007511,"(0,)"
3,2017-12-01,C_ID_d27d835a9f,2,1,0,36,33,'M_ID_44485d60c0',-19.435101,-0.539864,...,"(-1,)","(1,)","(7,)","(511,)",-0.104980,-0.052490,"(3,)",13.654417,6.827208,"(0,)"
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,116,53,'M_ID_d895cb84ae',37.205791,0.320740,...,"(17,)","(0,)","(37,)","(278,)",-0.047546,-0.047546,"(3,)",1.103070,1.103070,"(3,)"


In [221]:
del train['hist_merchant_id_<lambda>']
del test['hist_merchant_id_<lambda>']

In [222]:
train_final['merch_merchant_group_id_mode'] = train_final['merch_merchant_group_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_merchant_category_id_mode'] = train_final['merch_merchant_category_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_subsector_id_mode'] = train_final['merch_subsector_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_category_1_mode'] = train_final['merch_category_1_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_most_recent_sales_range_mode'] = train_final['merch_most_recent_sales_range_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_most_recent_purchases_range_mode'] = train_final['merch_most_recent_purchases_range_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_active_months_lag3_mode'] = train_final['merch_active_months_lag3_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_avg_sales_lag6_mode'] = train_final['merch_avg_sales_lag6_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_active_months_lag6_mode'] = train_final['merch_active_months_lag6_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_active_months_lag12_mode'] = train_final['merch_active_months_lag12_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_category_4_mode'] = train_final['merch_category_4_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_city_id_mode'] = train_final['merch_city_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_state_id_mode'] = train_final['merch_state_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train_final['merch_category_2_mode'] = train_final['merch_category_2_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))

In [223]:
test_final['merch_merchant_group_id_mode'] = test_final['merch_merchant_group_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_merchant_category_id_mode'] = test_final['merch_merchant_category_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_subsector_id_mode'] = test_final['merch_subsector_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_category_1_mode'] = test_final['merch_category_1_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_most_recent_sales_range_mode'] = test_final['merch_most_recent_sales_range_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_most_recent_purchases_range_mode'] = test_final['merch_most_recent_purchases_range_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_active_months_lag3_mode'] = test_final['merch_active_months_lag3_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_avg_sales_lag6_mode'] = test_final['merch_avg_sales_lag6_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_active_months_lag6_mode'] = test_final['merch_active_months_lag6_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_active_months_lag12_mode'] = test_final['merch_active_months_lag12_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_category_4_mode'] = test_final['merch_category_4_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_city_id_mode'] = test_final['merch_city_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_state_id_mode'] = test_final['merch_state_id_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test_final['merch_category_2_mode'] = test_final['merch_category_2_mode'].map(lambda x: str(x).lstrip('(').rstrip(',)'))

In [224]:
train_final.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,277,117,'M_ID_1a81c358a3',-175.047455,...,69,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1
1,2016-12-01,C_ID_59908e6dfe,3,1,1,-1.218335,235,88,'M_ID_1a81c358a3',-150.645996,...,69,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1
2,2017-08-01,C_ID_c685bb7b50,3,2,1,-0.938059,45,4,'M_ID_1a81c358a3',-32.290592,...,69,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1
3,2013-08-01,C_ID_69b673b48b,3,1,1,-1.514573,10,5,'M_ID_1a81c358a3',-7.382210,...,69,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1
4,2017-12-01,C_ID_3c883b3c2d,5,1,1,-1.195605,46,43,'M_ID_1a81c358a3',-15.803577,...,69,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1


In [225]:
test_final.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,...,merch_active_months_lag6_mode,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,...,69,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,...,69,"(0,)","(34,)","(560,)",40.468750,40.46875,3,1.06568204,1.06568204,0
2,2017-08-01,C_ID_b709037bc5,5,1,1,15,11,'M_ID_c708cbb89a',4.708528,0.313902,...,143,"(0,)","(33,)","(705,)",0.150757,0.1507568359375,3,1.00751115,1.00751115,0
3,2017-12-01,C_ID_d27d835a9f,2,1,0,36,33,'M_ID_44485d60c0',-19.435101,-0.539864,...,-1,"(1,)","(7,)","(511,)",-0.104980,-0.052490234375,3,13.65441663,6.827208315,0
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,116,53,'M_ID_d895cb84ae',37.205791,0.320740,...,17,"(0,)","(37,)","(278,)",-0.047546,-0.04754638671875,3,1.10307018,1.10307018,3


In [226]:
target = train_final['target']

In [ ]:
# y_train = train_final['target']
# del train_final['target']

In [227]:
train_final['month'] =  pd.DatetimeIndex(train_final['first_active_month']).month  
test_final['month'] = pd.DatetimeIndex(test_final['first_active_month']).month

del train_final['first_active_month']
del test_final['first_active_month']

In [228]:
test_card_ids = test_final['card_id']
del test_final['card_id']
del test_final['merchant_id']
del train_final['card_id']
del train_final['merchant_id']

In [ ]:
del test_final['old_index']

In [229]:
train_final.shape

(201917, 69)

In [230]:
test_final.shape

(123623, 69)

In [232]:
# y_train.shape

In [233]:
del train
del test
#del merchants_data
#del history

In [234]:
test_final.head()

,feature_1,feature_2,feature_3,hist_transactions_count,hist_merchant_id_nunique,hist_merchant_id_<lambda>,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,...,merch_avg_sales_lag12_sum,merch_avg_sales_lag12_mean,merch_avg_purchases_lag12_sum,merch_avg_purchases_lag12_mean,merch_active_months_lag12_mode,merch_category_4_mode,merch_city_id_mode,merch_state_id_mode,merch_category_2_mode,month
0,3,3,1,71,27,'M_ID_9139332ccc',-42.511078,-0.598748,0.235676,-0.743903,...,"(0,)","(34,)","(560,)",-0.017807,-0.0178070068359375,3,0.97945402,0.97945402,1,4.0
1,2,3,0,87,36,'M_ID_7e6e285f13',-55.080849,-0.633113,0.318817,-0.740723,...,"(0,)","(34,)","(560,)",40.468750,40.46875,3,1.06568204,1.06568204,0,1.0
2,5,1,1,15,11,'M_ID_c708cbb89a',4.708528,0.313902,2.525867,-0.724609,...,"(0,)","(33,)","(705,)",0.150757,0.1507568359375,3,1.00751115,1.00751115,0,8.0
3,2,1,0,36,33,'M_ID_44485d60c0',-19.435101,-0.539864,0.087965,-0.731881,...,"(1,)","(7,)","(511,)",-0.104980,-0.052490234375,3,13.65441663,6.827208315,0,12.0
4,5,1,1,116,53,'M_ID_d895cb84ae',37.205791,0.320740,15.782255,-0.746758,...,"(0,)","(37,)","(278,)",-0.047546,-0.04754638671875,3,1.10307018,1.10307018,3,12.0


In [235]:
test_final.isna().sum()

feature_1                                 0
feature_2                                 0
feature_3                                 0
hist_transactions_count                   0
hist_merchant_id_nunique                  0
hist_merchant_id_<lambda>                 0
hist_purchase_amount_sum                  0
hist_purchase_amount_mean                 0
hist_purchase_amount_max                  0
hist_purchase_amount_min                  0
hist_purchase_amount_std                  0
hist_city_id_nunique                      0
hist_city_id_<lambda>                     0
hist_merchant_category_id_nunique         0
hist_merchant_category_id_<lambda>        0
hist_category_2_sum                       0
hist_category_2_mean                      0
hist_category_2_<lambda>                  0
hist_state_id_nunique                     0
hist_state_id_<lambda>                    0
hist_category_3_sum                       0
hist_category_3_mean                      0
hist_category_3_<lambda>        

In [236]:
test_final['month'].fillna(12.0, inplace = True)

In [ ]:
x_train = np.array(train_final)
x_test = np.array(test_final)
y_train = np.array(y_train).reshape(201917,1)

In [240]:
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# set default parameters for 1st round training
params = {'boosting': 'gbdt',
          'objective':'regression',
          'metric': 'rmse',
          'learning_rate': 0.01, # 0.003! #0.005 #0.006 
          'num_leaves': 110, #110 #100 #150 large, but over-fitting
          'max_bin': 66,  #60 #50 # large,but slower,over-fitting
          'max_depth': 10, # deal with over-fitting
          'min_data_in_leaf': 30, # deal with over-fitting
          'min_child_samples': 20,
          'feature_fraction': 0.5,#0.5 #0.6 #0.8
          'bagging_fraction': 0.8,
          'bagging_freq': 40,#5  
          'bagging_seed': 11,
          'lambda_l1': 2,#1.3! #5 #1.2 #1
          'lambda_l2': 0.1 #0.1
         }

In [247]:
train_final = train_final.select_dtypes(exclude=['object'])
test_final = test_final.select_dtypes(exclude=['object'])

In [ ]:
# Reference: code from Ashish Patel(阿希什)Repeated KFOLD Approach: RMSE[3.70]
# Kfold cross-validation
# folds = KFold(n_splits=5, shuffle=True, random_state=11)

nfolds = 5
nrepeats = 2 
folds = RepeatedKFold(n_splits=nfolds, n_repeats=nrepeats, random_state=11)
fold_pred = np.zeros(len(train_final))
feature_importance_df = pd.DataFrame()
lgb_preds = np.zeros(len(test_final))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_final.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_final.iloc[trn_idx], label=target.iloc[trn_idx]) #categorical_feature=categorical_feats
    val_data = lgb.Dataset(train_final.iloc[val_idx], label=target.iloc[val_idx]) #categorical_feature=categorical_feats

    iteration = 2000
    lgb_m = lgb.train(params, trn_data, iteration, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    fold_pred[val_idx] = lgb_m.predict(train_final.iloc[val_idx], num_iteration=lgb_m.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_final.columns
    fold_importance_df["importance"] = lgb_m.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    lgb_preds += lgb_m.predict(test_final, num_iteration=lgb_m.best_iteration) / (nfolds*nrepeats)

print("CV score: {:<8.5f}".format(np.sqrt(mean_squared_error(fold_pred, target))))

fold n°0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 2.53869	valid_1's rmse: 2.59931
[200]	training's rmse: 1.86595	valid_1's rmse: 1.98175
[300]	training's rmse: 1.41081	valid_1's rmse: 1.59817
[400]	training's rmse: 1.23383	valid_1's rmse: 1.47195
[500]	training's rmse: 1.12652	valid_1's rmse: 1.41463
[600]	training's rmse: 1.06614	valid_1's rmse: 1.39289
[700]	training's rmse: 1.01437	valid_1's rmse: 1.38089
[800]	training's rmse: 0.972155	valid_1's rmse: 1.37671
[900]	training's rmse: 0.937871	valid_1's rmse: 1.37638
[1000]	training's rmse: 0.909413	valid_1's rmse: 1.37696
Early stopping, best iteration is:
[948]	training's rmse: 0.921946	valid_1's rmse: 1.3754
fold n°1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 2.54474	valid_1's rmse: 2.57551
[200]	training's rmse: 1.8706	valid_1's rmse: 1.95852
[300]	training's rmse: 1.41892	valid_1's rmse: 1.56578
[400]	training's rmse: 1.2431	valid_1's rmse: 1.